In [4]:
import os
Root = "C:\\Users\\Hp\\speech-emotion-recognition-ravdess-data"
os.chdir(Root)

In [6]:
import librosa
import soundfile
import os, glob, pickle # saving the model in Pickle format
import numpy as np # For array
from sklearn.model_selection import train_test_split
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import accuracy_score

In [7]:
#Feature Extraction from Audios
def extract_feature(file_name, mfcc, chroma, mel):
    with soundfile.SoundFile(file_name) as sound_file:
        X = sound_file.read(dtype="float32")
        sample_rate=sound_file.samplerate
        if chroma:
            stft=np.abs(librosa.stft(X))
        result=np.array([])
        if mfcc:
            mfccs=np.mean(librosa.feature.mfcc(y=X, sr=sample_rate, n_mfcc=40).T, axis=0)
            result=np.hstack((result, mfccs))
        if chroma:
            chroma=np.mean(librosa.feature.chroma_stft(S=stft, sr=sample_rate).T,axis=0)
            result=np.hstack((result, chroma))
        if mel:
            mel=np.mean(librosa.feature.melspectrogram(X, sr=sample_rate).T,axis=0)
            result=np.hstack((result, mel))
    return result

In [8]:
# We have following 8 emotions in Ravdess dataset
emotions={
  '01':'neutral',
  '02':'calm',
  '03':'happy',
  '04':'sad',
  '05':'angry',
  '06':'fearful',
  '07':'disgust',
  '08':'surprised'
}

#4 Types of emotions that we will observe
observed_emotions=['calm', 'happy', 'fearful', 'disgust']

In [20]:
#Load the data and extract features for each sound file
def load_data(test_size=0.2):
    x,y=[],[]
    for file in glob.glob("C:\\Users\\Hp\\speech-emotion-recognition-ravdess-data\\Actor_*/*.wav"):
        file_name=os.path.basename(file)
        emotion=emotions[file_name.split("-")[2]]
        if emotion not in observed_emotions:
            continue
        feature=extract_feature(file, mfcc=True, chroma=True, mel=True)
        x.append(feature)
        y.append(emotion)
    return train_test_split(np.array(x), y, test_size=test_size, random_state=9)

In [21]:
#Split the dataset
x_train,x_test,y_train,y_test=load_data(test_size=0.25)

<ipython-input-7-138437059bc8>:16: FutureWarning: Pass y=[0. 0. 0. ... 0. 0. 0.] as keyword args. From version 0.10 passing these as positional arguments will result in an error
  mel=np.mean(librosa.feature.melspectrogram(X, sr=sample_rate).T,axis=0)
<ipython-input-7-138437059bc8>:16: FutureWarning: Pass y=[0.0000000e+00 3.0517578e-05 3.0517578e-05 ... 0.0000000e+00 0.0000000e+00
 0.0000000e+00] as keyword args. From version 0.10 passing these as positional arguments will result in an error
  mel=np.mean(librosa.feature.melspectrogram(X, sr=sample_rate).T,axis=0)
<ipython-input-7-138437059bc8>:16: FutureWarning: Pass y=[ 0.0000000e+00  0.0000000e+00 -3.0517578e-05 ... -3.0517578e-05
 -3.0517578e-05 -3.0517578e-05] as keyword args. From version 0.10 passing these as positional arguments will result in an error
  mel=np.mean(librosa.feature.melspectrogram(X, sr=sample_rate).T,axis=0)
<ipython-input-7-138437059bc8>:16: FutureWarning: Pass y=[3.0517578e-05 3.0517578e-05 3.0517578e-05 ... 

In [22]:
x_train

array([[-5.22061890e+02,  3.50668907e+01,  3.75342917e+00, ...,
         1.65243153e-04,  1.04321596e-04,  6.55571566e-05],
       [-6.41227722e+02,  4.49487762e+01, -1.85174119e+00, ...,
         3.89261913e-05,  3.05255380e-05,  2.94166657e-05],
       [-6.50705750e+02,  5.30211639e+01, -4.92040443e+00, ...,
         4.75216839e-05,  3.46632478e-05,  1.62844426e-05],
       ...,
       [-5.50096191e+02,  1.70297680e+01, -1.14575634e+01, ...,
         1.51764631e-04,  1.16828531e-04,  8.47479314e-05],
       [-5.55357605e+02,  4.71569710e+01,  1.10750742e+01, ...,
         1.61086457e-04,  1.04962470e-04,  6.52811723e-05],
       [-5.04816345e+02,  3.53618660e+01, -1.43495789e+01, ...,
         6.08151488e-04,  5.55269769e-04,  4.47782193e-04]])

In [23]:
#Get the number of features extracted
print(f'Features extracted: {x_train.shape[1]}')

Features extracted: 180


In [24]:
#Initialize the Multi Layer Perceptron Classifier
model=MLPClassifier(alpha=0.01, batch_size=256, epsilon=1e-08, hidden_layer_sizes=(300,), learning_rate='adaptive', max_iter=500)

In [25]:
#Train the model
model.fit(x_train,y_train)

MLPClassifier(alpha=0.01, batch_size=256, hidden_layer_sizes=(300,),
              learning_rate='adaptive', max_iter=500)

In [26]:
#Predict for the test set
y_pred=model.predict(x_test)

In [27]:
y_pred

array(['happy', 'fearful', 'happy', 'happy', 'fearful', 'calm', 'calm',
       'disgust', 'calm', 'happy', 'happy', 'fearful', 'fearful', 'happy',
       'disgust', 'happy', 'calm', 'disgust', 'disgust', 'calm',
       'disgust', 'disgust', 'disgust', 'calm', 'happy', 'happy',
       'fearful', 'happy', 'fearful', 'fearful', 'happy', 'fearful',
       'happy', 'fearful', 'happy', 'calm', 'calm', 'fearful', 'calm',
       'calm', 'happy', 'calm', 'calm', 'calm', 'fearful', 'calm',
       'disgust', 'happy', 'calm', 'happy', 'fearful', 'fearful', 'calm',
       'happy', 'happy', 'fearful', 'calm', 'happy', 'calm', 'calm',
       'disgust', 'calm', 'happy', 'calm', 'disgust', 'calm', 'calm',
       'disgust', 'fearful', 'happy', 'fearful', 'fearful', 'fearful',
       'fearful', 'fearful', 'disgust', 'fearful', 'happy', 'fearful',
       'fearful', 'disgust', 'calm', 'fearful', 'calm', 'disgust',
       'happy', 'disgust', 'fearful', 'happy', 'fearful', 'disgust',
       'fearful', 'calm'

In [28]:
#Calculate the accuracy of our model
accuracy=accuracy_score(y_true=y_test, y_pred=y_pred)

#Print the accuracy
print("Accuracy: {:.2f}%".format(accuracy*100))

Accuracy: 67.19%


In [29]:
from sklearn.metrics import accuracy_score, f1_score

In [30]:
f1_score(y_test, y_pred,average=None)

array([0.8       , 0.60240964, 0.58947368, 0.65934066])

In [31]:
#Creating a dataframe
import pandas as pd
df=pd.DataFrame({'Actual': y_test, 'Predicted':y_pred})
df.head(20)

,Actual,Predicted
0,happy,happy
1,calm,fearful
2,happy,happy
3,happy,happy
4,disgust,fearful
5,calm,calm
6,happy,calm
7,happy,disgust
8,disgust,calm
9,happy,happy


In [32]:
import pickle
with open( 'modelForPrediction1.sav', 'wb') as f:
    pickle.dump(model,f)

In [33]:
filename = 'modelForPrediction1.sav'
loaded_model = pickle.load(open(filename, 'rb')) 

feature=extract_feature("C:\\Users\\Hp\\speech-emotion-recognition-ravdess-data\\Actor_01\\03-01-06-02-01-01-01.wav", mfcc=True, chroma=True, mel=True)

feature=feature.reshape(1,-1)

prediction=loaded_model.predict(feature)
prediction

<ipython-input-7-138437059bc8>:16: FutureWarning: Pass y=[5.1879883e-04 2.4414062e-04 1.5258789e-04 ... 0.0000000e+00 0.0000000e+00
 6.1035156e-05] as keyword args. From version 0.10 passing these as positional arguments will result in an error
  mel=np.mean(librosa.feature.melspectrogram(X, sr=sample_rate).T,axis=0)


array(['fearful'], dtype='<U7')

In [35]:
filename = 'modelForPrediction1.sav'
loaded_model = pickle.load(open(filename, 'rb')) 

feature=extract_feature("C:\\Users\\Hp\\speech-emotion-recognition-ravdess-data\\Actor_01\\03-01-03-02-01-02-01.wav", mfcc=True, chroma=True, mel=True)

feature=feature.reshape(1,-1)

prediction=loaded_model.predict(feature)
prediction

<ipython-input-7-138437059bc8>:16: FutureWarning: Pass y=[0. 0. 0. ... 0. 0. 0.] as keyword args. From version 0.10 passing these as positional arguments will result in an error
  mel=np.mean(librosa.feature.melspectrogram(X, sr=sample_rate).T,axis=0)


array(['happy'], dtype='<U7')